In [ ]:
# This file aims to display some functions regarding Brightway 2.5, to demonstrate what can be done with the software when you have a foreground model matched with ecoinvent's bio- and technosphere.

# Analysis

### 1. Loading the project

In [ ]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

In [ ]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

### 2. Database overview

In [ ]:
bd.databases

In [ ]:
bd.projects.current

In [ ]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
db_template = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

### 3. Database exploration

In [ ]:
db_template.search('ceramics')

In [ ]:
ei_clca.search('smartphone')

In [22]:
# define a process as object in the project
ceramics = db_template.get(name = 'porcelain ceramics production', location = "RER", unit = 'kilogram')
smartphone = ei_clca.get(name = 'consumer electronics production, mobile device, smartphone', location = "GLO", unit = 'unit')


In [ ]:
# quickly print all exchanges of the process defined above, sorted by amount (large to small)
exchanges = sorted(smartphone.exchanges(), key=lambda exc: exc['amount'], reverse=True)
for exc in exchanges:
    print(exc)

### 4. Impact assessment methods

In [ ]:
# here we will analyse the set of methods that are available as part of the background
list(bd.methods)[:5]

In [ ]:
# printing all methods that contain a specific keyword, e.g. "climate change"
climate_methods = [m for m in bd.methods if any("climate change" in str(part).lower() for part in m)]
for method in climate_methods:
    print(method)

In [ ]:
# selecting the impact assessment methods that we want to use in our analysis
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')

### 4. LCI 

In [ ]:
# export the LCI to e
bi.export.excel.write_lci_excel(db_template.name,
                                objs=[ceramics],
                                dirpath=Path.cwd())

### 5. LCIA

In [ ]:
# Quick LCIA calculation
smartphone_lca = smartphone.lca(lcia_gwp100)
smartphone_lca.score


In [23]:
ceramics_lca = ceramics.lca(lcia_gwp100)
ceramics_lca.score

0.7265791393172016

### 6. Contribution analysis

In [ ]:
# One option to do a contribution analysis is to use the recursive calculation method
bwa.print_recursive_calculation(smartphone,
lcia_method=lcia_gwp100,max_level=3,cutoff=0.05)

In [ ]:
# Another option is to use the recursive calculation to an object, which returns a DataFrame
smartphone_ca_gwp = bwa.utils.recursive_calculation_to_object(smartphone,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.02,
                                          as_dataframe=True,
                                          )
smartphone_ca_gwp

In [ ]:
# Elementary flows contribution analysis by process
smartphone_lca.to_dataframe().pivot_table(index=['col_name', 'row_name'], values='amount', aggfunc='sum').sort_values(by='amount', ascending=False)

In [ ]:
# Elementary flows contribution analysis as summary table
smartphone_lca.to_dataframe().pivot_table(index='row_name',values='amount',aggfunc='sum')

### 7. Plot Contribution analysis graphs

In [ ]:
# Filter out the parent processes that are not relevant for the contribution analysis
smartphone_ca_gwp = smartphone_ca_gwp.dropna(subset='parent') 



In [ ]:
f, ax = plt.subplots(figsize=(6, 3))
sns.barplot(y='name', x='score', data=smartphone_ca_gwp, ax=ax)
ax.set(
    title='Contribution analysis of the product under study',
    xlabel='CO2 eq. emissions (kg)',
    ylabel='')
plt.tight_layout()
plt.show()


### 8. Comparison

In [24]:
combined_df = pd.concat([smartphone_lca.set_index('name').score,ceramics_lca.set_index('name').score],
                        keys=['smartphone','ceramics'],
                        names=['activity','contributor'])
combined_df

AttributeError: 'LCA' object has no attribute 'set_index'